In [ ]:
import os
import math
import datetime

import numpy as np
import xarray as xr
import netCDF4 as nc
import proplot as pplt
import geocat.viz as gv
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import cartopy.feature as cfeature

from enum import IntEnum
from matplotlib.animation import FuncAnimation

from wrf import to_np


In [ ]:
class FigureScope(IntEnum):
    LOCAL=0
    CONUS=1

In [ ]:
try:
    import IPython
    __file__ = IPython.extract_module_locals()[1]["__vsc_ipynb_file__"]
    root_dir = os.path.dirname(__file__)
except (AttributeError, ImportError):
    root_dir = os.path.abspath("")

data_root = os.path.join(root_dir, 'data')

In [ ]:
Denver = (-104.9903, 39.7392)

#Colorado Lat Lon
#-109.5,-101.8, 36.5, 41.5

#CONUS Lat Lon
#-124,-65, 25, 49

In [ ]:
def plot_snow_acc(colo_ds_2d=None, colo_ds_3d=None, colo_domain=None, date_time=None, output=False, show=False, **kwargs):
    # Plot Snow accululation for 3_16 through 3_19
    # Plot Snow -- Colorado
    
    colo_lat = colo_domain.variables['XLAT'][:]
    colo_lon = colo_domain.variables['XLONG'][:]
    colo_hgt = colo_domain.variables['HGT'][:]
    
    # Generate figure (set its size (width, height) in inches)
    fig = plt.figure(figsize=(18.5, 10.5))

    # Generate axes using Cartopy
    ax = plt.axes(projection=ccrs.PlateCarree())

    ax.set_extent((-109.5,-101.8, 36.5, 41.5), crs=ccrs.PlateCarree())
    ax.add_feature(cfeature.STATES)


    # Add filled contours
    _cont = plt.contourf(
        to_np(colo_lon),
        to_np(colo_lat),
        # colo_ds_2d.SNOW_ACC_NC.mean('Time'),
        colo_ds_2d.PREC_ACC_NC.mean('Time'),
        # colo_ds_3d.GS_LWC.mean('Time'),
        # levels=np.linspace(-16,5, 21),
        # cmap="Ice_r",
        cmap='YlGnBu',
        # vmin=-16,
        # vmax=5,
        zorder=4,
        alpha = 0.8
    )

    _topo = plt.contour(
        to_np(colo_lon),
        to_np(colo_lat),
        colo_hgt[0],
        alpha = 0.6,
        cmap ='gray',
        # vmin=5500,
        # vmax=5500,
        levels = 30
    )

    ax.clabel(_topo, inline = 1, fontsize = 6)

    # Add a colorbar
    _cbar = plt.colorbar(
        _cont,
        orientation="vertical",
        # ticks=np.arange(-16,10, 1),
        drawedges=True,
        extendrect=True,
        shrink=0.9
    )


    # Format colorbar ticks and labels
    _cbar.set_label('Accumulation (mm)', fontsize=12)

    # Draw gridlines
    _gl = ax.gridlines(
        crs=ccrs.PlateCarree(),
        draw_labels=True,
        dms=False,
        x_inline=False,
        y_inline=False,
        linewidth=1,
        color="k",
        alpha=0.25,
        zorder=4
    )

    # Manipulate latitude and longitude gridline numbers and spacing
    _gl.top_labels = False
    _gl.right_labels = False

    _gl.xlabel_style = {"rotation": 45, "size": 10}
    _gl.ylabel_style = {"rotation": 0, "size": 10}
    _gl.xlines = True
    _gl.ylines = True

    # Add titles and labels to projection
    gv.set_titles_and_labels(
        ax,
        maintitle=f"{date_time:%Y} Colorado Snowstorm - Local Snow Accumulation {date_time:%B} {date_time:%d} ",
        maintitlefontsize=14
    )

    plt.plot(*Denver, 'ko', markersize=5, zorder=5)
    plt.text(*Denver, ' Denver', fontsize=12, zorder=6)

    #ax.legend()
    fig.patch.set_facecolor('white')

    if output:
        output_file=f"SNOW_ACC_{date_time:%Y}{date_time:%m}{date_time:%d}.png"
        output_path=os.path.join(root_dir, 'figures', output_file)
        plt.savefig(output_path, dpi=300, transparent=False)
    if show:
        plt.show()
        plt.close()


In [ ]:
# plot_snow_acc(
#     xr.open_dataset(os.path.join(data_root, 'Colorado_2D_SV_03_16.nc'), engine = 'netcdf4'),
#     xr.open_dataset(os.path.join(data_root, 'Colorado_3D_SV_03_16.nc'), engine = 'netcdf4'),
#     nc.Dataset(os.path.join(data_root, 'wrfconstants_usgs404_Colorado.nc')),
#     datetime.datetime(2003,3,16),
#     show=True
#     )

In [ ]:
def plot_local_temp(colo_ds_3d=None, colo_domain=None, date_time=None, output=False, show=False, **kwargs):
    # Plot Temperature -- Colorado
    
    colo_lat = colo_domain.variables['XLAT'][:]
    colo_lon = colo_domain.variables['XLONG'][:]
    colo_hgt = colo_domain.variables['HGT'][:]
    
    # Generate figure (set its size (width, height) in inches)
    fig = plt.figure(figsize=(18.5, 10.5))

    # Generate axes using Cartopy
    ax = plt.axes(projection=ccrs.PlateCarree())

    ax.set_extent((-109.5,-101.8, 36.5, 41.5), crs=ccrs.PlateCarree())
    ax.add_feature(cfeature.STATES)


    # Add filled contours
    _cont = plt.contourf(
        to_np(colo_lon),
        to_np(colo_lat),
        colo_ds_3d.AS_Tc.mean('Time'),
        levels=np.linspace(-15, 0, 21),
        # cmap="Ice_r",
        cmap='YlGnBu',
        vmin=-16,
        vmax=0,
        zorder=4,
        alpha = 0.8)

    _topo = plt.contour(
        to_np(colo_lon),
        to_np(colo_lat),
        colo_hgt[0],
        alpha = 0.6,
        cmap = 'gray',
        # vmin=5500,
        # vmax=5500,
        levels = 30
    )

    ax.clabel(_topo, inline = 1, fontsize = 6)

    # Add a colorbar
    _cbar = plt.colorbar(
        _cont,
        orientation="vertical",
        ticks=np.arange(-15, 2, 1),
        drawedges=True,
        extendrect=True,
        shrink=0.9
    )


    # Format colorbar ticks and labels
    _cbar.ax.tick_params(size=1, labelsize=10)
    _cbar.set_label('Temperature (C)',fontsize = 12)

    # Draw gridlines
    _gl = ax.gridlines(
        crs=ccrs.PlateCarree(),
        draw_labels=True,
        dms=False,
        x_inline=False,
        y_inline=False,
        linewidth=1,
        color="k",
        alpha=0.25,
        zorder=4
    )

    # Manipulate latitude and longitude gridline numbers and spacing
    _gl.top_labels = False
    _gl.right_labels = False

    _gl.xlabel_style = {"rotation": 45, "size": 10}
    _gl.ylabel_style = {"rotation": 0, "size": 10}
    _gl.xlines = True
    _gl.ylines = True

    # Add titles and labels to projection
    gv.set_titles_and_labels(
        ax,
        maintitle=f"{date_time:%Y} Colorado Snowstorm - Local Temperature {date_time:%B} {date_time:%d} ",
        maintitlefontsize=14
    )

    plt.plot(*Denver, 'ko', markersize=5, zorder=5)
    plt.text(*Denver, ' Denver', fontsize=12, zorder=6)

    fig.patch.set_facecolor('white')
    if output:
        output_file=f"COLO_TEMP_{date_time:%Y}{date_time:%m}{date_time:%d}.png"
        output_path=os.path.join(root_dir, 'figures', output_file)
        plt.savefig(output_path, dpi=300, transparent=False)
    if show:
        plt.show()
        plt.close()

In [ ]:
# plot_local_temp(
#     xr.open_dataset(os.path.join(data_root, 'Colorado_3D_SV_03_16.nc'), engine = 'netcdf4'),
#     nc.Dataset(os.path.join(data_root, 'wrfconstants_usgs404_Colorado.nc')),
#     datetime.datetime(2003,3,16),
#     show=True
# )

# animate_temp(
#     ds_3d=xr.open_dataset(os.path.join(data_root, 'Colorado_3D_SV_03_16.nc'), engine = 'netcdf4'), 
#     domain=nc.Dataset(os.path.join(data_root, 'wrfconstants_usgs404_Colorado.nc')),
#     scope=FigureScope.LOCAL)

# animate_temp(
#     ds_3d=xr.open_dataset(os.path.join(data_root, 'CONUS_3D_SV_03_16.nc'), engine = 'netcdf4'),
#     domain=nc.Dataset(os.path.join(data_root, 'wrfconstants_usgs404_CONUS.nc')),
#     scope=FigureScope.CONUS)

In [ ]:
def plot_conus_temp(conus_ds_3d=None, conus_domain=None, date_time=None, output=False, show=False, **kwargs):
    #Plot Temperature -- CONUS
    
    conus_lat = conus_domain.variables['XLAT'][:]
    conus_lon = conus_domain.variables['XLONG'][:]
    conus_hgt = conus_domain.variables['HGT'][:]
    
    # Generate figure (set its size (width, height) in inches)
    fig = plt.figure(figsize=(25.5, 10.5))

    # Generate axes using Cartopy
    ax = plt.axes(projection=ccrs.PlateCarree())

    ax.set_extent((-130, -65, 25, 50), crs=ccrs.PlateCarree())
    ax.add_feature(cfeature.STATES)


    # Add filled contours
    _cont = plt.contourf(
        to_np(conus_lon),
        to_np(conus_lat),
        conus_ds_3d.AS_Tc.mean('Time'),
        levels=np.linspace(-30, 5, 21),
        # cmap="Ice_r",
        cmap='YlGnBu',
        vmin=-30,
        vmax=5,
        zorder=4,
        alpha = 0.6
    )

    _topo = plt.contour(
        to_np(conus_lon),
        to_np(conus_lat),
        conus_hgt[0],
        alpha = 0.3,
        cmap = 'gray',
        # vmin=5500,
        # vmax=5500,
        levels = 100
    )

    # ax.clabel(_topo, inline = 1, fontsize = 6)

    # Add a colorbar
    _cbar = plt.colorbar(
        _cont,
        orientation="vertical",
        ticks=np.arange(-30, 6, 3),
        drawedges=True,
        extendrect=True,
        shrink=0.9
    )


    # Format colorbar ticks and labels
    _cbar.ax.tick_params(size=1, labelsize=10)
    _cbar.set_label('Temperature (C)',fontsize = 12)

    # Draw gridlines
    _gl = ax.gridlines(
        crs=ccrs.PlateCarree(),
        draw_labels=True,
        dms=False,
        x_inline=False,
        y_inline=False,
        linewidth=1,
        color="k",
        alpha=0.25,
        zorder=4
    )

    # Manipulate latitude and longitude gridline numbers and spacing
    _gl.top_labels = False
    _gl.right_labels = False

    _gl.xlabel_style = {"rotation": 45, "size": 10}
    _gl.ylabel_style = {"rotation": 0, "size": 10}
    _gl.xlines = True
    _gl.ylines = True

    # Add titles and labels to projection
    gv.set_titles_and_labels(
        ax,
        maintitle=f"{date_time:%Y} Colorado Snowstorm - CONUS Temperature {date_time:%B} {date_time:%d} ",
        maintitlefontsize=14
    )

    plt.plot(*Denver, 'ko', markersize=5, zorder=5)
    plt.text(*Denver, ' Denver', fontsize=12, zorder=6)

    fig.patch.set_facecolor('white')
    if output:
        output_file=f"CONUS_TEMP_{date_time:%Y}{date_time:%m}{date_time:%d}.png"
        output_path=os.path.join(root_dir, 'figures', output_file)
        plt.savefig(output_path, dpi=300, transparent=False)
    if show:
        plt.show()
        plt.close()

In [ ]:
# plot_conus_temp(
#     xr.open_dataset(os.path.join(data_root, 'CONUS_3D_SV_03_16.nc'), engine = 'netcdf4'),
#     nc.Dataset(os.path.join(data_root, 'wrfconstants_usgs404_CONUS.nc')),
#     datetime.datetime(2003,3,16),
#     show=True
# )

In [ ]:
def plot_conus_wind(conus_ds_3d=None, conus_domain=None, date_time=None, output=False, show=False, **kwargs):
    # Plot Wind Contours

    conus_lat = conus_domain.variables['XLAT'][:]
    conus_lon = conus_domain.variables['XLONG'][:]
    conus_hgt = conus_domain.variables['HGT'][:]
    
    uwind = conus_ds_3d.U_500MB
    vwind = conus_ds_3d.V_500MB

    wspd = np.sqrt(uwind**2+vwind**2)
    wdir = np.arctan2(uwind.values,vwind.values)*180/math.pi+180
    
    # Generate figure (set its size (width, height) in inches)
    fig = plt.figure(figsize=(25.5, 10.5))

    # Generate axes using Cartopy
    ax = plt.axes(projection=ccrs.PlateCarree())

    ax.set_extent((-130, -65, 25, 50), crs=ccrs.PlateCarree())
    ax.add_feature(cfeature.STATES)


    # Add filled contours
    _wspd=plt.contourf(
        to_np(conus_lon),
        to_np(conus_lat),
        wspd.mean('Time'), 
        levels=np.linspace(0, 60, 21),
        cmap='YlGnBu',
        vmin=0,
        vmax=60,
        zorder=4,
        alpha = 0.6
    )

    _topo = plt.contour(
        to_np(conus_lon),
        to_np(conus_lat),
        conus_hgt[0],
        alpha = 0.3,
        cmap ='gray',
        # vmin=5500,
        # vmax=5500,
        levels = 100
    )

    # ax.clabel(_topo, inline = 1, fontsize = 6)

    # Add a colorbar
    _cbar = plt.colorbar(
        _wspd,
        label='Wind Speed (m/s)',
        orientation="vertical",
        ticks=np.arange(0, 65, 5),
        drawedges=True,
        extendrect=True,
        shrink=0.9)

    _q=plt.quiver(
        to_np(conus_lon)[::20, ::20], 
        to_np(conus_lat)[::20, ::20], 
        uwind.mean('Time')[::20, ::20], 
        vwind.mean('Time')[::20, ::20], 
        scale=4500, 
        # units='width'
    )
    plt.quiverkey(_q, 0.7, 0.8, 30, r'$30 \frac{m}{s}$', labelpos='E', coordinates='figure')

    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.title('Wind Contours')


    # Format colorbar ticks and labels
    _cbar.ax.tick_params(size=1, labelsize=10)
    _cbar.set_label('Wind Speed',fontsize = 12)

    # Draw gridlines
    _gl = ax.gridlines(
        crs=ccrs.PlateCarree(),
        draw_labels=True,
        dms=False,
        x_inline=False,
        y_inline=False,
        linewidth=1,
        color="k",
        alpha=0.25,
        zorder=4
    )

    # Manipulate latitude and longitude gridline numbers and spacing
    _gl.top_labels = False
    _gl.right_labels = False

    _gl.xlabel_style = {"rotation": 45, "size": 10}
    _gl.ylabel_style = {"rotation": 0, "size": 10}
    _gl.xlines = True
    _gl.ylines = True

    # Add titles and labels to projection
    gv.set_titles_and_labels(
        ax,
        maintitle=f"{date_time:%Y} Colorado Snowstorm - CONUS Winds {date_time:%B} {date_time:%d} ",
        maintitlefontsize=14
    )

    plt.plot(*Denver, 'ko', markersize=5, zorder=5)
    plt.text(*Denver, ' Denver', fontsize=12, zorder=6)

    #ax.legend()
    fig.patch.set_facecolor('white')
    if output:
        output_file=f"CONUS_WIND_{date_time:%Y}{date_time:%m}{date_time:%d}.png"
        output_path=os.path.join(root_dir, 'figures', output_file)
        plt.savefig(output_path, dpi=300, transparent=False)
    if show:
        plt.show()
        plt.close()

In [ ]:
# plot_conus_wind(
#     xr.open_dataset(os.path.join(data_root, 'CONUS_3D_SV_03_16.nc'), engine = 'netcdf4'),
#     nc.Dataset(os.path.join(data_root, 'wrfconstants_usgs404_CONUS.nc')),
#     datetime.datetime(2003,3,16),
#     show=True
# )

In [ ]:
def plot_local_wind(colo_ds_3d=None, colo_domain=None, date_time=None, output=False, show=False, **kwargs):
    # Plot Wind Contours

    colo_lat = colo_domain.variables['XLAT'][:]
    colo_lon = colo_domain.variables['XLONG'][:]
    colo_hgt = colo_domain.variables['HGT'][:]
    
    uwind = colo_ds_3d.U_500MB
    vwind = colo_ds_3d.V_500MB

    wspd = np.sqrt(uwind**2+vwind**2)
    wdir = np.arctan2(uwind.values,vwind.values)*180/math.pi+180
    
    # Generate figure (set its size (width, height) in inches)
    fig = plt.figure(figsize=(25.5, 10.5))

    # Generate axes using Cartopy
    ax = plt.axes(projection=ccrs.PlateCarree())

    ax.set_extent((-109.5,-101.8, 36.5, 41.5), crs=ccrs.PlateCarree())
    ax.add_feature(cfeature.STATES)


    # Add filled contours
    _wspd=plt.contourf(
        to_np(colo_lon),
        to_np(colo_lat),
        wspd.mean('Time'), 
        levels=np.linspace(0, 25, 21),
        cmap='YlGnBu',
        # vmin=0,
        # vmax=60,
        zorder=4,
        alpha = 0.6
    )

    _topo = plt.contour(
        to_np(colo_lon),
        to_np(colo_lat),
        colo_hgt[0],
        alpha = 0.3,
        cmap ='gray',
        # vmin=5500,
        # vmax=5500,
        levels = 30
    )

    ax.clabel(_topo, inline = 1, fontsize = 6)

    # Add a colorbar
    _cbar = plt.colorbar(
        _wspd,
        label='Wind Speed (m/s)',
        orientation="vertical",
        ticks=np.arange(0, 26, 2),
        drawedges=True,
        extendrect=True,
        shrink=0.9)

    _q=plt.quiver(
        to_np(colo_lon)[::5, ::5], 
        to_np(colo_lat)[::5, ::5], 
        uwind.mean('Time')[::5, ::5], 
        vwind.mean('Time')[::5, ::5], 
        scale=500, 
        # units='width'
    )
    plt.quiverkey(_q, 0.8, 0.9, 10, r'$5 \frac{m}{s}$', labelpos='E', coordinates='figure')

    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.title('Wind Contours')


    # Format colorbar ticks and labels
    _cbar.ax.tick_params(size=1, labelsize=10)
    _cbar.set_label('Wind Speed',fontsize = 12)

    # Draw gridlines
    _gl = ax.gridlines(
        crs=ccrs.PlateCarree(),
        draw_labels=True,
        dms=False,
        x_inline=False,
        y_inline=False,
        linewidth=1,
        color="k",
        alpha=0.25,
        zorder=4
    )

    # Manipulate latitude and longitude gridline numbers and spacing
    _gl.top_labels = False
    _gl.right_labels = False

    _gl.xlabel_style = {"rotation": 45, "size": 10}
    _gl.ylabel_style = {"rotation": 0, "size": 10}
    _gl.xlines = True
    _gl.ylines = True

    # Add titles and labels to projection
    gv.set_titles_and_labels(
        ax,
        maintitle=f"{date_time:%Y} Colorado Snowstorm - CONUS Winds {date_time:%B} {date_time:%d} ",
        maintitlefontsize=14
    )

    plt.plot(*Denver, 'ko', markersize=5, zorder=5)
    plt.text(*Denver, ' Denver', fontsize=12, zorder=6)

    #ax.legend()
    fig.patch.set_facecolor('white')
    if output:
        output_file=f"COLO_WIND_{date_time:%Y}{date_time:%m}{date_time:%d}.png"
        output_path=os.path.join(root_dir, 'figures', output_file)
        plt.savefig(output_path, dpi=300, transparent=False)
    if show:
        plt.show()
        plt.close()

In [ ]:
# plot_local_wind(
#     xr.open_dataset(os.path.join(data_root, 'Colorado_3D_SV_03_16.nc'), engine = 'netcdf4'),
#     nc.Dataset(os.path.join(data_root, 'wrfconstants_usgs404_Colorado.nc')),
#     datetime.datetime(2003,3,16),
#     show=True
# )

In [ ]:
show_plots = False
save_figures = True

colo_domain = nc.Dataset(os.path.join(data_root, 'wrfconstants_usgs404_Colorado.nc'))
conus_domain = nc.Dataset(os.path.join(data_root, 'wrfconstants_usgs404_CONUS.nc'))

_days=[
    {
        'date_time':datetime.datetime(2003,3,16), 
        'colo_ds_2d':xr.open_dataset(os.path.join(data_root, 'Colorado_2D_SV_03_16.nc'), engine = 'netcdf4'),
        'colo_ds_3d':xr.open_dataset(os.path.join(data_root, 'Colorado_3D_SV_03_16.nc'), engine = 'netcdf4'),
        'colo_domain':colo_domain,
        'conus_ds_2d':xr.open_dataset(os.path.join(data_root, 'CONUS_2D_SV_03_16.nc'), engine = 'netcdf4'),
        'conus_ds_3d':xr.open_dataset(os.path.join(data_root, 'CONUS_3D_SV_03_16.nc'), engine = 'netcdf4'),
        'conus_domain':conus_domain,
        'show':show_plots,
        'output':save_figures
    },
    {
        'date_time':datetime.datetime(2003,3,17), 
        'colo_ds_2d':xr.open_dataset(os.path.join(data_root, 'Colorado_2D_SV_03_17.nc'), engine = 'netcdf4'),
        'colo_ds_3d':xr.open_dataset(os.path.join(data_root, 'Colorado_3D_SV_03_17.nc'), engine = 'netcdf4'),
        'colo_domain':colo_domain,
        'conus_ds_2d':xr.open_dataset(os.path.join(data_root, 'CONUS_2D_SV_03_17.nc'), engine = 'netcdf4'),
        'conus_ds_3d':xr.open_dataset(os.path.join(data_root, 'CONUS_3D_SV_03_17.nc'), engine = 'netcdf4'),
        'conus_domain':conus_domain,
        'show':show_plots,
        'output':save_figures
    },
    {
        'date_time':datetime.datetime(2003,3,18), 
        'colo_ds_2d':xr.open_dataset(os.path.join(data_root, 'Colorado_2D_SV_03_18.nc'), engine = 'netcdf4'),
        'colo_ds_3d':xr.open_dataset(os.path.join(data_root, 'Colorado_3D_SV_03_18.nc'), engine = 'netcdf4'),
        'colo_domain':colo_domain,
        'conus_ds_2d':xr.open_dataset(os.path.join(data_root, 'CONUS_2D_SV_03_18.nc'), engine = 'netcdf4'),
        'conus_ds_3d':xr.open_dataset(os.path.join(data_root, 'CONUS_3D_SV_03_18.nc'), engine = 'netcdf4'),
        'conus_domain':conus_domain,
        'show':show_plots,
        'output':save_figures
    },
    {
        'date_time':datetime.datetime(2003,3,19), 
        'colo_ds_2d':xr.open_dataset(os.path.join(data_root, 'Colorado_2D_SV_03_19.nc'), engine = 'netcdf4'),
        'colo_ds_3d':xr.open_dataset(os.path.join(data_root, 'Colorado_3D_SV_03_19.nc'), engine = 'netcdf4'),
        'colo_domain':colo_domain,
        'conus_ds_2d':xr.open_dataset(os.path.join(data_root, 'CONUS_2D_SV_03_19.nc'), engine = 'netcdf4'),
        'conus_ds_3d':xr.open_dataset(os.path.join(data_root, 'CONUS_3D_SV_03_19.nc'), engine = 'netcdf4'),
        'conus_domain':conus_domain,
        'show':show_plots,
        'output':save_figures
    },
]
    



In [ ]:
for _day in _days:
    plot_snow_acc(**_day)
    plot_local_temp(**_day)
    plot_conus_temp(**_day)
    plot_conus_wind(**_day)
    plot_local_wind(**_day)


In [ ]:
def animate_wind(ds_3d=None, domain=None, scope=None, **kwargs):
    # Plot Wind Contours

    lat = domain.variables['XLAT'][:]
    lon = domain.variables['XLONG'][:]
    hgt = domain.variables['HGT'][:]
    
    uwind = ds_3d.U_500MB
    vwind = ds_3d.V_500MB

    wspd = np.sqrt(uwind**2+vwind**2)
    wdir = np.arctan2(uwind.values,vwind.values)*180/math.pi+180

    def get_data(ii):
        return wspd[ii], uwind[ii], vwind[ii]
    
    z, u, v = get_data(0)

    data_limits=(math.ceil(wspd.min())-1, math.ceil(wspd.max()))
    data_buff=int((max(data_limits)-min(data_limits))/10)
    stagger=5 if scope == FigureScope.LOCAL else 20
    qscale=500 if scope == FigureScope.LOCAL else 4500
    date_time=datetime.datetime.utcfromtimestamp( int((ds_3d.Time[0] - np.datetime64('1970-01-01T00:00:00')) / np.timedelta64(1, 's')) )
    start=date_time
    end=datetime.datetime.utcfromtimestamp( int((ds_3d.Time[-1] - np.datetime64('1970-01-01T00:00:00')) / np.timedelta64(1, 's')) )
    if scope == FigureScope.LOCAL:
        fig = plt.figure(figsize=(7.5, 4))
    else:
        fig = plt.figure(figsize=(12, 4))
    ax = plt.axes(projection=ccrs.PlateCarree())
    if scope == FigureScope.LOCAL:
        ax.set_extent((-109.5,-101.8, 36.5, 41.5), crs=ccrs.PlateCarree())
    else:
        ax.set_extent((-130, -65, 25, 50), crs=ccrs.PlateCarree())
    ax.add_feature(cfeature.STATES)
    if scope == FigureScope.LOCAL:
        scope_name=scope.name.capitalize()
    else:
        scope_name=scope.name
    title=ax.set_title(f"{date_time:%Y} Colorado Snowstorm - {scope_name} Wind Contours {date_time:%B} {date_time:%d} {date_time:%H}:{date_time:%M}:{date_time:%S} ", fontsize=14)

    cont=[plt.contourf(
        to_np(lon),
        to_np(lat),
        z, 
        levels=np.linspace(min(data_limits), max(data_limits), 21),
        cmap='YlGnBu',
        zorder=7,
        alpha = 0.6)]
    _topo = plt.contour(
        to_np(lon),
        to_np(lat),
        hgt[0],
        alpha = 0.4,
        linewidths=0.3,
        zorder=3,
        cmap ='gray',
        levels = 100)
    _cbar = plt.colorbar(
        cont[0],
        label='Wind Speed (m/s)',
        orientation="vertical",
        ticks=np.arange(min(data_limits), max(data_limits)+data_buff, data_buff  ),
        drawedges=True,
        extendrect=True,
        shrink=0.9)

    _q=plt.quiver(
        to_np(lon)[::stagger, ::stagger], 
        to_np(lat)[::stagger, ::stagger], 
        u[::stagger, ::stagger], 
        v[::stagger, ::stagger], 
        scale=qscale, 
        zorder=4,
        alpha=0.7,
        animated=True
    )

    if scope == FigureScope.CONUS:
        plt.quiverkey(_q, 0.8, 0.9, 30, r'$30 \frac{m}{s}$', labelpos='E', coordinates='figure')
    else:
        plt.quiverkey(_q, 0.853, 0.9, 10, r'$5 \frac{m}{s}$', labelpos='E', coordinates='figure')

    _cbar.ax.tick_params(size=1, labelsize=10)
    _cbar.set_label('Wind Speed',fontsize = 12)
    _gl = ax.gridlines(
        crs=ccrs.PlateCarree(),
        draw_labels=True,
        dms=False,
        x_inline=False,
        y_inline=False,
        linewidth=1,
        color="k",
        alpha=0.25,
        zorder=4)
    _gl.top_labels = False
    _gl.right_labels = False
    _gl.xlabel_style = {"rotation": 45, "size": 10}
    _gl.ylabel_style = {"rotation": 0, "size": 10}
    _gl.xlines = True
    _gl.ylines = True


    plt.plot(*Denver, 'ko', markersize=3, zorder=4)
    plt.text(*Denver, ' Denver', fontsize=8, zorder=4)

    def animate(ii):
        date_time=datetime.datetime.utcfromtimestamp( int((ds_3d.Time[ii] - np.datetime64('1970-01-01T00:00:00')) / np.timedelta64(1, 's')) )
        _=[c.remove() for c in cont[0].collections]
        z, u, v = get_data(ii)
        cont[0] = ax.contourf(to_np(lon), to_np(lat), z, cmap='YlGnBu', zorder=4 ,alpha = 0.8)
        _q.set_UVC(u[::stagger, ::stagger], v[::stagger, ::stagger])
        title.set_text(f"{date_time:%Y} Colorado Snowstorm - {scope_name} Wind Contours {date_time:%B} {date_time:%d} {date_time:%H}:{date_time:%M}:{date_time:%S} ")
        return cont[0].collections+[title, _q]

    # ani = FuncAnimation(fig, animate, frames=len(ds_3d.Time), interval=50, blit=True, repeat=True)
    ani = FuncAnimation(fig, animate, frames=len(ds_3d.Time), blit=True, repeat=True)
    plt.show()
    ani.save(f'WIND_{scope.name}_{start:%m}{start:%d}{start:%H}_{end:%m}{end:%d}{end:%H}.gif')

In [ ]:
def animate_temp(ds_3d=None, domain=None, scope=None, **kwargs):
    # Animate Temperature
    
    lat = domain.variables['XLAT'][:]
    lon = domain.variables['XLONG'][:]
    hgt = domain.variables['HGT'][:]
    
    def get_data(ii):
        return ds_3d.AS_Tc[ii]
    
    z = get_data(0)
    data_limits=(math.ceil(ds_3d.AS_Tc.min())-1, math.ceil(ds_3d.AS_Tc.max()))
    data_buff=int((max(data_limits)-min(data_limits))/10)
    date_time=datetime.datetime.utcfromtimestamp( int((ds_3d.Time[0] - np.datetime64('1970-01-01T00:00:00')) / np.timedelta64(1, 's')) )
    start=date_time
    end=datetime.datetime.utcfromtimestamp( int((ds_3d.Time[-1] - np.datetime64('1970-01-01T00:00:00')) / np.timedelta64(1, 's')) )
    if scope == FigureScope.LOCAL:
        fig = plt.figure(figsize=(7.5, 4))
    else:
        fig = plt.figure(figsize=(12, 4))
    ax = plt.axes(projection=ccrs.PlateCarree())
    if scope == FigureScope.LOCAL:
        ax.set_extent((-109.5,-101.8, 36.5, 41.5), crs=ccrs.PlateCarree())
    else:
        ax.set_extent((-130, -65, 25, 50), crs=ccrs.PlateCarree())
    ax.add_feature(cfeature.STATES)
    if scope == FigureScope.LOCAL:
        scope_name=scope.name.capitalize()
    else:
        scope_name=scope.name
    title=ax.set_title(f"{date_time:%Y} Colorado Snowstorm - {scope_name} Temperature {date_time:%B} {date_time:%d} {date_time:%H}:{date_time:%M}:{date_time:%S} ", fontsize=14)
    cont = [plt.contourf(
        to_np(lon),
        to_np(lat),
        z,
        levels=np.linspace(min(data_limits), max(data_limits), 21),
        cmap='YlGnBu',
        zorder=4,
        alpha = 0.8)]
    _topo = plt.contour(
        to_np(lon),
        to_np(lat),
        hgt[0],
        alpha = 0.3,
        linewidths=0.5,
        cmap = 'gray',
        levels = 100)
    _cbar = plt.colorbar(
        cont[0],
        orientation="vertical",
        ticks=np.arange(min(data_limits), max(data_limits)+data_buff, data_buff  ),
        drawedges=True,
        extendrect=True,
        shrink=0.9)
    _cbar.ax.tick_params(size=1, labelsize=10)
    _cbar.set_label('Temperature (C)',fontsize = 12)
    _gl = ax.gridlines(
        crs=ccrs.PlateCarree(),
        draw_labels=True,
        dms=False,
        x_inline=False,
        y_inline=False,
        linewidth=1,
        color="k",
        alpha=0.25,
        zorder=4)
    _gl.top_labels = False
    _gl.right_labels = False
    _gl.xlabel_style = {"rotation": 45, "size": 10}
    _gl.ylabel_style = {"rotation": 0, "size": 10}
    _gl.xlines = True
    _gl.ylines = True
    plt.plot(*Denver, 'ko', markersize=3, zorder=5)
    plt.text(*Denver, ' Denver', fontsize=8, zorder=6)
    fig.patch.set_facecolor('white')
    
    def animate(ii):
        date_time=datetime.datetime.utcfromtimestamp( int((ds_3d.Time[ii] - np.datetime64('1970-01-01T00:00:00')) / np.timedelta64(1, 's')) )
        _=[c.remove() for c in cont[0].collections]
        cont[0] = ax.contourf(to_np(lon), to_np(lat), get_data(ii), cmap='YlGnBu', zorder=4 ,alpha = 0.8)
        title.set_text(f"{date_time:%Y} Colorado Snowstorm - {scope_name} Temperature {date_time:%B} {date_time:%d} {date_time:%H}:{date_time:%M}:{date_time:%S}")
        return cont[0].collections+[title]

    ani = FuncAnimation(fig, animate, frames=len(ds_3d.Time), blit=True, repeat=True)
    plt.show()
    ani.save(f'TEMP_{scope.name}_{start:%m}{start:%d}{start:%H}_{end:%m}{end:%d}{end:%H}.gif')

In [ ]:
def animate_geopotential(ds_2d=None, domain=None, scope=None, **kwargs):
    # Animate Geopotential
    
    lat = domain.variables['XLAT'][:]
    lon = domain.variables['XLONG'][:]
    hgt = domain.variables['HGT'][:]
    
    def get_data(ii):
        return ds_2d.Z[ii]
    
    z = get_data(0)
    data_limits=(math.ceil(ds_2d.AS_Tc.min())-1, math.ceil(ds_2d.AS_Tc.max()))
    data_buff=int((max(data_limits)-min(data_limits))/10)
    date_time=datetime.datetime.utcfromtimestamp( int((ds_2d.Time[0] - np.datetime64('1970-01-01T00:00:00')) / np.timedelta64(1, 's')) )
    start=date_time
    end=datetime.datetime.utcfromtimestamp( int((ds_2d.Time[-1] - np.datetime64('1970-01-01T00:00:00')) / np.timedelta64(1, 's')) )
    if scope == FigureScope.LOCAL:
        fig = plt.figure(figsize=(7.5, 4))
    else:
        fig = plt.figure(figsize=(12, 4))
    ax = plt.axes(projection=ccrs.PlateCarree())
    if scope == FigureScope.LOCAL:
        ax.set_extent((-109.5,-101.8, 36.5, 41.5), crs=ccrs.PlateCarree())
    else:
        ax.set_extent((-130, -65, 25, 50), crs=ccrs.PlateCarree())
    ax.add_feature(cfeature.STATES)
    if scope == FigureScope.LOCAL:
        scope_name=scope.name.capitalize()
    else:
        scope_name=scope.name
    title=ax.set_title(f"{date_time:%Y} Colorado Snowstorm - {scope_name} Geopotential Height {date_time:%B} {date_time:%d} {date_time:%H}:{date_time:%M}:{date_time:%S} ", fontsize=14)
    cont = [plt.contourf(
        to_np(lon),
        to_np(lat),
        z,
        levels=np.linspace(min(data_limits), max(data_limits), 21),
        cmap='YlGnBu',
        zorder=4,
        alpha = 0.8)]
    _topo = plt.contour(
        to_np(lon),
        to_np(lat),
        hgt[0],
        alpha = 0.3,
        linewidths=0.5,
        cmap = 'gray',
        levels = 100)
    _cbar = plt.colorbar(
        cont[0],
        orientation="vertical",
        ticks=np.arange(min(data_limits), max(data_limits)+data_buff, data_buff  ),
        drawedges=True,
        extendrect=True,
        shrink=0.9)
    _cbar.ax.tick_params(size=1, labelsize=10)
    _cbar.set_label('Geopotential Height (meters MSL)',fontsize = 12)
    _gl = ax.gridlines(
        crs=ccrs.PlateCarree(),
        draw_labels=True,
        dms=False,
        x_inline=False,
        y_inline=False,
        linewidth=1,
        color="k",
        alpha=0.25,
        zorder=4)
    _gl.top_labels = False
    _gl.right_labels = False
    _gl.xlabel_style = {"rotation": 45, "size": 10}
    _gl.ylabel_style = {"rotation": 0, "size": 10}
    _gl.xlines = True
    _gl.ylines = True
    plt.plot(*Denver, 'ko', markersize=3, zorder=5)
    plt.text(*Denver, ' Denver', fontsize=8, zorder=6)
    fig.patch.set_facecolor('white')
    
    def animate(ii):
        date_time=datetime.datetime.utcfromtimestamp( int((ds_2d.Time[ii] - np.datetime64('1970-01-01T00:00:00')) / np.timedelta64(1, 's')) )
        _=[c.remove() for c in cont[0].collections]
        cont[0] = ax.contourf(to_np(lon), to_np(lat), get_data(ii), cmap='YlGnBu', zorder=4 ,alpha = 0.8)
        title.set_text(f"{date_time:%Y} Colorado Snowstorm - {scope_name} Temperature {date_time:%B} {date_time:%d} {date_time:%H}:{date_time:%M}:{date_time:%S}")
        return cont[0].collections+[title]

    ani = FuncAnimation(fig, animate, frames=len(ds_2d.Time), blit=True, repeat=True)
    plt.show()
    ani.save(f'Z_{scope.name}_{start:%m}{start:%d}{start:%H}_{end:%m}{end:%d}{end:%H}.gif')

In [ ]:
def animate_snow_acc(ds_2d=None, domain=None, scope=None, **kwargs):
    # Plot Snow accululation for 3_16 through 3_19
    
    lat = domain.variables['XLAT'][:]
    lon = domain.variables['XLONG'][:]
    hgt = domain.variables['HGT'][:]
    
    def get_data(ii):
        return ds_2d.PREC_ACC_NC[ii]
    
    z = get_data(0)
    data_limits=(math.ceil(ds_2d.PREC_ACC_NC.min())-1, math.ceil(ds_2d.PREC_ACC_NC.max()))
    data_buff=int((max(data_limits)-min(data_limits))/10)
    date_time=datetime.datetime.utcfromtimestamp( int((ds_2d.Time[0] - np.datetime64('1970-01-01T00:00:00')) / np.timedelta64(1, 's')) )
    start=date_time
    end=datetime.datetime.utcfromtimestamp( int((ds_2d.Time[-1] - np.datetime64('1970-01-01T00:00:00')) / np.timedelta64(1, 's')) )
    if scope == FigureScope.LOCAL:
        fig = plt.figure(figsize=(7.5, 4))
    else:
        fig = plt.figure(figsize=(12, 4))
    ax = plt.axes(projection=ccrs.PlateCarree())
    if scope == FigureScope.LOCAL:
        ax.set_extent((-109.5,-101.8, 36.5, 41.5), crs=ccrs.PlateCarree())
    else:
        ax.set_extent((-130, -65, 25, 50), crs=ccrs.PlateCarree())

    ax.add_feature(cfeature.STATES)
    if scope == FigureScope.LOCAL:
        scope_name=scope.name.capitalize()
    else:
        scope_name=scope.name
    title = ax.set_title(f"{date_time:%Y} Colorado Snowstorm - {scope_name} Snow Accumulation {date_time:%B} {date_time:%d} {date_time:%H}:{date_time:%M}:{date_time:%S}", fontsize=14)
    cont = [ plt.contourf(
        to_np(lon), 
        to_np(lat), 
        z, 
        levels=np.linspace(min(data_limits), max(data_limits), 21),
        cmap='YlGnBu', 
        zorder=4, 
        alpha = 0.8) ]
    _topo = plt.contour(
        to_np(lon),
        to_np(lat),
        hgt[0],
        alpha = 0.3,
        linewidths=0.5,
        cmap ='gray',
        levels = 100)
    # ax.clabel(_topo, inline = 1, fontsize = 3)
    _cbar = plt.colorbar(
        cont[0],
        orientation="vertical",
        ticks=np.arange(min(data_limits), max(data_limits)+data_buff, data_buff  ),
        drawedges=True,
        extendrect=True,
        shrink=0.9)
    _cbar.set_label('Accumulation (mm)', fontsize=12)
    _gl = ax.gridlines(
        crs=ccrs.PlateCarree(),
        draw_labels=True,
        dms=False,
        x_inline=False,
        y_inline=False,
        linewidth=1,
        color="k",
        alpha=0.25,
        zorder=4)
    _gl.top_labels = False
    _gl.right_labels = False
    _gl.xlabel_style = {"rotation": 45, "size": 10}
    _gl.ylabel_style = {"rotation": 0, "size": 10}
    _gl.xlines = True
    _gl.ylines = True
    plt.plot(*Denver, 'ko', markersize=3, zorder=5)
    plt.text(*Denver, ' Denver', fontsize=8, zorder=6)
    fig.patch.set_facecolor('white')
    def animate(ii):
        date_time=datetime.datetime.utcfromtimestamp( int((ds_2d.Time[ii] - np.datetime64('1970-01-01T00:00:00')) / np.timedelta64(1, 's')) )
        # _=[c.remove() for c in cont[0].collection]
        cont[0] = ax.contourf(to_np(lon), to_np(lat), get_data(ii), cmap='YlGnBu', zorder=4 ,alpha = 0.8)
        title.set_text(f"{date_time:%Y} Colorado Snowstorm - {scope_name} Snow Accumulation {date_time:%B} {date_time:%d} {date_time:%H}:{date_time:%M}:{date_time:%S}")
        return cont[0].collections+[title]

    ani = FuncAnimation(fig, animate, frames=len(ds_2d.Time), blit=True, repeat=True)
    plt.show()
    ani.save(f'SNOW_ACC_{scope.name}_{start:%m}{start:%d}{start:%H}_{end:%m}{end:%d}{end:%H}.gif')

In [ ]:
# animate_snow_acc(
#     ds_2d=xr.open_dataset(os.path.join(data_root, 'Colorado_2D_SV_03.nc'), engine = 'netcdf4'), 
#     domain=nc.Dataset(os.path.join(data_root, 'wrfconstants_usgs404_Colorado.nc')),
#     scope=FigureScope.LOCAL)

In [ ]:
# animate_snow_acc(
#     ds_2d=xr.open_dataset(os.path.join(data_root, 'Colorado_2D_SV_03.nc'), engine = 'netcdf4'), 
#     domain=nc.Dataset(os.path.join(data_root, 'wrfconstants_usgs404_Colorado.nc')),
#     scope=FigureScope.LOCAL)

# animate_wind(
#     ds_3d=xr.open_dataset(os.path.join(data_root, 'Colorado_3D_SV_03.nc'), engine = 'netcdf4'),
#     domain=nc.Dataset(os.path.join(data_root, 'wrfconstants_usgs404_Colorado.nc')),
#     scope=FigureScope.LOCAL)

# animate_wind(
#     ds_3d=xr.open_dataset(os.path.join(data_root, 'CONUS_3D_SV_03.nc'), engine = 'netcdf4'),
#     domain=nc.Dataset(os.path.join(data_root, 'wrfconstants_usgs404_CONUS.nc')),
#     scope=FigureScope.CONUS)

# animate_temp(
#     ds_3d=xr.open_dataset(os.path.join(data_root, 'CONUS_3D_SV_03.nc'), engine = 'netcdf4'),
#     domain=nc.Dataset(os.path.join(data_root, 'wrfconstants_usgs404_CONUS.nc')),
#     scope=FigureScope.CONUS)

# animate_temp(
#     ds_3d=xr.open_dataset(os.path.join(data_root, 'Colorado_3D_SV_03.nc'), engine = 'netcdf4'),
#     domain=nc.Dataset(os.path.join(data_root, 'wrfconstants_usgs404_Colorado.nc')),
#     scope=FigureScope.LOCAL)

# animate_geopotential(
#     ds_2d=xr.open_dataset(os.path.join(data_root, 'Colorado_2D_SV_03.nc'), engine = 'netcdf4'), 
#     domain=nc.Dataset(os.path.join(data_root, 'wrfconstants_usgs404_Colorado.nc')),
#     scope=FigureScope.LOCAL)

# animate_geopotential(
#     ds_2d=xr.open_dataset(os.path.join(data_root, 'CONUS_2D_SV_03.nc'), engine = 'netcdf4'), 
#     domain=nc.Dataset(os.path.join(data_root, 'wrfconstants_usgs404_CONUS.nc')),
#     scope=FigureScope.CONUS)